In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from nltk.stem.wordnet import WordNetLemmatizer
from pdfminer.converter import TextConverter
from nltk.tokenize import word_tokenize
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from nltk.corpus import stopwords
from collections import Counter
from cStringIO import StringIO
import collections
import warnings
import pickle
import nltk
warnings.filterwarnings("ignore")

In [2]:
def pdfToTxt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
def removeStopWordsLemmatize(text):
    stop_words = set(stopwords.words('english'))
    to_remove = ['\n','&', '(', ')','.','\x80', '\x93', '\x96', '\x9c', 
             '\x9e', '\x9f', '\xa4', '\xb6', '\xbc', '\xc3', '\xe2','$',
             ':', ';', '?','~', '\x81', '\x82', '\x83', '\x84', '\x86',
             '\x87', '\x88', '\x89', '\x8b', '\x8c', '\x90', '\x91', '\x92',
             '\x94', '\x97', '\x98', '\x99', '\x9b', '\x9d', '\xa0', '\xa1',
             '\xa2', '\xa3', '\xa5', '\xa6', '\xa7', '\xa8', '\xa9', '\xaa',
             '\xab', '\xac', '\xad', '\xae', '\xaf', '\xb0', '\xb1', '\xb2',
             '\xb3', '\xb4', '\xb5', '\xb7', '\xb8', '\xb9', '\xba', '\xbb'
             , '\xbd', '\xbe', '\xbf', '\xc2', '\xc4', '\xc5', '\xc9', '\xcb',
             '\xce', '\xcf', '\xd0', '\xe1', '\xef', '=', '\\', '^', '!', '#',
             '%', '+', ',', '-', '/','\x8a', '\x9a', '\xca','"','{','}','|',"'","  ",'would']
    stop_words.update(to_remove)
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if w not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(t) for t in filtered_sentence]
    return lem

In [4]:
def saveObj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
def loadObj(name):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
def createSentences(text):
    stri = StringIO(str(text.encode('utf-8')))
    sentences = []
    while True:
        nl = stri.readline()
        if nl != '':
            sentences.append(nl)
        else:
            break
    return sentences

In [7]:
def getSentenceScore(sentences, word_rank):
    d = {}
    sentence_score = 0
    c = 0
    for sentence in sentences:
        sentence_score = 0
        for word in sentence:
            if word in word_rank:
                sentence_score += len(word_rank) - word_rank.index(word)
        d[c] = sentence_score
        c += 1
    return d

In [8]:
def getSummery(compression_ratio,ordered_dic,sentences):
    sentencesToInclude = (compression_ratio*len(sentences))//100
    compressed = list()
    counter = 1
    for i in ordered_dic:
        compressed.append(sentences[i])
        if counter == sentencesToInclude:
            break
        counter+=1
    return compressed